In [ ]:
import paramiko
import asyncio
from typing import AsyncGenerator, List, Dict, Optional
from dataclasses import dataclass
import re

@dataclass
class OutputChunk:
    text: str
    is_complete: bool = False

@dataclass
class CommandContext:
    command: str
    output: str = ""
    thoughts: List[str] = None
    is_complete: bool = False
    
    def __post_init__(self):
        if self.thoughts is None:
            self.thoughts = []

class ContextManager:
    def __init__(self, max_history: int = 5):
        self.history: List[CommandContext] = []
        self.max_history = max_history
        self.current_context: Optional[CommandContext] = None
    
    def start_command(self, command: str) -> CommandContext:
        self.current_context = CommandContext(command=command)
        return self.current_context
    
    def add_output(self, output: str):
        if self.current_context:
            self.current_context.output += output
    
    def add_thought(self, thought: str):
        if self.current_context:
            self.current_context.thoughts.append(thought)
    
    def complete_command(self):
        if self.current_context:
            self.current_context.is_complete = True
            self.history.append(self.current_context)
            
            # Maintain history size
            if len(self.history) > self.max_history:
                self.history = self.history[-self.max_history:]
            
            self.current_context = None
    
    def get_recent_context(self, count: int = 2) -> List[CommandContext]:
        return self.history[-count:] if self.history else []


class PatternMatcher:
    """Detects patterns in SSH output that might indicate when to interrupt or continue"""
    
    def __init__(self):
        # Common patterns that indicate command completion or prompt for input
        self.prompt_patterns = [
            re.compile(r'[$#>]\s*$'),  # Common shell prompts
            re.compile(r'password:', re.IGNORECASE),  # Password prompts
            re.compile(r'\[y/n\]', re.IGNORECASE),  # Yes/no confirmation
            re.compile(r'continue\?', re.IGNORECASE),  # Continue prompts
        ]
        
        # Patterns that indicate ongoing processes
        self.ongoing_patterns = [
            re.compile(r'^\s*\d+%', re.MULTILINE),  # Progress indicators
            re.compile(r'loading', re.IGNORECASE),
            re.compile(r'please wait', re.IGNORECASE),
        ]
    
    def should_interact(self, text: str) -> bool:
        """Determine if we should interact based on the output"""
        for pattern in self.prompt_patterns:
            if pattern.search(text):
                return True
        return False
    
    def is_ongoing_process(self, text: str) -> bool:
        """Check if the output indicates an ongoing process"""
        for pattern in self.ongoing_patterns:
            if pattern.search(text):
                return True
        return False


class StreamProcessor:
    def __init__(self, matcher: PatternMatcher):
        self.matcher = matcher
        self.buffer = ""
        self.output_line_count = 0
        
    async def process_stream(self, stream_generator) -> AsyncGenerator[OutputChunk, None]:
        async for data in stream_generator:
            self.buffer += data
            
            # Check if we should yield this chunk
            should_interact = self.matcher.should_interact(self.buffer)
            is_ongoing = self.matcher.is_ongoing_process(self.buffer)
            
            # Count output lines for line-based decision making
            new_line_count = self.buffer.count('\n')
            
            # Decision logic for when to yield chunks
            should_yield = (
                should_interact or 
                (new_line_count - self.output_line_count >= 10 and not is_ongoing) or
                len(self.buffer) > 1000  # Yield after a certain amount of content
            )
            
            if should_yield:
                yield OutputChunk(
                    text=self.buffer,
                    is_complete=should_interact
                )
                self.output_line_count = new_line_count
                self.buffer = ""


class SSHConnection_Paramiko:
    def __init__(self):
        self.client = paramiko.SSHClient()
        self.client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.client.connect(
            hostname=ssh_jump_dest,
            username=ssh_username,
            pkey=private_key,
            sock=channel)
        
        # Initialize the context manager
        self.context_manager = ContextManager()
        
    async def stream_output(self, shell) -> AsyncGenerator[str, None]:
        """Stream output from shell with backpressure control"""
        while True:
            if shell.recv_ready():
                data = shell.recv(4096).decode('utf-8')
                if data:
                    yield data
            await asyncio.sleep(0.1)
            
    async def execute_interactive(self, command: str) -> AsyncGenerator[OutputChunk, None]:
        """Execute command and stream output chunks"""
        shell = self.client.invoke_shell()
        processor = StreamProcessor(PatternMatcher())
        
        # Start a new command context
        context = self.context_manager.start_command(command)
        
        # Send command
        shell.send(command + '\n')
        
        # Process output stream
        async for chunk in processor.process_stream(self.stream_output(shell)):
            # Update the context with the output
            self.context_manager.add_output(chunk.text)
            
            # If the command appears to be complete, mark it accordingly
            if chunk.is_complete:
                self.context_manager.complete_command()
                
            yield chunk
            
    async def execute_with_llm(self, command: str, llm_handler):
        """Execute a command and use LLM to decide on further interactions"""
        shell = self.client.invoke_shell()
        processor = StreamProcessor(PatternMatcher())
        
        # Start a new command context
        context = self.context_manager.start_command(command)
        
        # Send initial command
        shell.send(command + '\n')
        
        # Get recent context to provide to the LLM
        recent_context = self.context_manager.get_recent_context()
        
        # Continuously process output
        buffer = ""
        async for data in self.stream_output(shell):
            buffer += data
            self.context_manager.add_output(data)
            
            # Check if we need LLM to make a decision
            if processor.matcher.should_interact(buffer):
                # Prepare context for LLM
                llm_input = {
                    "current_command": command,
                    "current_output": buffer,
                    "recent_history": [
                        {"command": ctx.command, "output": ctx.output} 
                        for ctx in recent_context
                    ],
                    "thoughts": context.thoughts
                }
                
                # Get LLM decision
                llm_response = await llm_handler(llm_input)
                
                # Store LLM reasoning as thought
                if "thought" in llm_response:
                    self.context_manager.add_thought(llm_response["thought"])
                
                # Handle LLM decision
                if "action" in llm_response:
                    action = llm_response["action"]
                    
                    if action["type"] == "continue":
                        # Just continue monitoring
                        buffer = ""
                    elif action["type"] == "respond":
                        # Send a response
                        response_text = action["text"]
                        shell.send(response_text + '\n')
                        buffer = ""
                    elif action["type"] == "complete":
                        # Mark command as complete and exit
                        self.context_manager.complete_command()
                        break
            
            # Also yield after accumulating significant output
            if len(buffer) > 1000:
                buffer = ""